### This notebook shows how to update a Dataiku Knowledge Bank using Dataiku Python API. In this example, we detect the new files being added through checking whether the files are uploaded after a certain time that was recorded as a project variable. 

In [0]:
%pylab inline

In [1]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [1]:
#from datetime import datetime, timedelta
import dataiku
import time

In [2]:
# Fetch variables
project = dataiku.api_client().get_default_project()
variables = project.get_variables()


# Create range of dates
now = int(time.time()*1000) #datetime.now().strftime("20%y-%m-%d")
variables["standard"]["datetime_upload"] = now
project.set_variables(variables)


In [3]:
now

1706160888196

In [4]:
type(now)

int

In [2]:
project = dataiku.api_client().get_default_project()
variables = project.get_variables()
click_time = variables["standard"]["datetime_upload"]
click_time

1706238003659

In [3]:
folder = dataiku.Folder('pdfs')
contents = folder.get_path_details()
for i in contents["children"]:
    epoch_time=i["lastModified"] 
    print(epoch_time)

1705986819086
1706074298626
1706240592602


In [4]:
uploaded_file_list = []
for i in contents["children"]:
    epoch_time=i["lastModified"]
    if epoch_time > click_time:
        #print(epoch_time - now)
        filename = i["fullPath"]
        uploaded_file_list.append(filename)
    
print(uploaded_file_list)

['/third-quarter-2023-performance-and-risk-monitoring-report.pdf']


In [14]:
### Getting the knowledge bank ID
kb_id = project.list_knowledge_banks()[0]['id']
kb = dataiku.KnowledgeBank(kb_id)
# Read recipe inputs
chunks = dataiku.Dataset("risk_text_1")
all_texts = chunks.get_dataframe()
print(kb_id)


IPfgEibh


In [15]:
all_texts.head()

,chunk_id,error_message,file,metadata,text
0,1,NaN,/first-quarter-2023-performance-risk-monitorin...,{'page': 1},Find more information on Fundsingapore.com 1\n...
1,5,NaN,/first-quarter-2023-performance-risk-monitorin...,{'page': 5},Find more information on Fundsingapore.com 5\n...
2,9,NaN,/first-quarter-2023-performance-risk-monitorin...,{'page': 9},Find more information on Fundsingapore.com 9\n...
3,13,NaN,/first-quarter-2023-performance-risk-monitorin...,{'page': 13},Find more information on Fundsingapore.com 13\...
4,17,NaN,/first-quarter-2023-performance-risk-monitorin...,{'page': 17},Find more information on Fundsingapore.com 17\...


In [22]:
new_texts = all_texts[all_texts['file'].isin(uploaded_file_list)]

In [23]:
new_texts = new_texts.reset_index(drop=True)

In [24]:
new_texts.head()

,chunk_id,error_message,file,metadata,text
0,1,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 1},Find more information on Fundsingapore.com 1\n...
1,5,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 5},Find more information on Fundsingapore.com 5\n...
2,9,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 9},Find more information on Fundsingapore.com 9\n...
3,13,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 13},Find more information on Fundsingapore.com 13\...
4,17,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 17},Find more information on Fundsingapore.com 17\...


In [25]:
def filepage(row):
    #return row['file']+'- page ' + str(row['chunk_id'])
    return row['file']+ ' ' + row['metadata'] +' ' + row['text']

In [26]:
### We concantenate the text and source metadata together 
### as the knowledge bank can only be updated using a single column
### it can't take in metadata columns separately
new_texts['text_and_source'] = new_texts.apply(filepage, axis=1)

In [27]:
new_texts.head()

,chunk_id,error_message,file,metadata,text,text_and_source
0,1,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 1},Find more information on Fundsingapore.com 1\n...,/third-quarter-2023-performance-and-risk-monit...
1,5,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 5},Find more information on Fundsingapore.com 5\n...,/third-quarter-2023-performance-and-risk-monit...
2,9,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 9},Find more information on Fundsingapore.com 9\n...,/third-quarter-2023-performance-and-risk-monit...
3,13,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 13},Find more information on Fundsingapore.com 13\...,/third-quarter-2023-performance-and-risk-monit...
4,17,NaN,/third-quarter-2023-performance-and-risk-monit...,{'page': 17},Find more information on Fundsingapore.com 17\...,/third-quarter-2023-performance-and-risk-monit...


In [28]:
#new_texts_df = new_texts[['source','text']]
#new_texts_df = new_texts[['file','metadata','text']]
new_texts_df = new_texts[['text_and_source']]

In [29]:
from langchain.document_loaders import DataFrameLoader
loader = DataFrameLoader(new_texts_df, page_content_column="text_and_source")
documents = loader.load()

In [30]:
vs = kb.as_langchain_vectorstore()
vs.add_documents(documents)

distutils Version classes are deprecated. Use packaging.version instead.
distutils Version classes are deprecated. Use packaging.version instead.


['056686cf-7c7b-4806-af03-9c5898174bbf',
 '746bef92-ffff-4615-b2c5-3c2ea5fb57f6',
 '99958ded-0eed-4c1e-bf43-2b46b5403bac',
 '31e8b56d-a9a9-45ea-b449-99d8d7ac3513',
 '4d913e25-0d0b-44d6-858d-37aa8663ce7d',
 '8c309cbb-6649-4a86-a07c-44ede9401bd0',
 '80f407e4-71f2-40e7-8c38-cb07dc25ec66',
 '4c3c4eef-1ea5-4a3e-a711-763012e552ac',
 'de1803b7-dc0c-43be-9230-e1a9bd13709d',
 'bc629248-048b-481a-b25a-1627b8db1648',
 'b5966003-307f-4d0d-ad69-cbc51882e83a',
 '09e09382-87ff-4537-a1b4-f9d2d7c78ddb',
 '9cc57290-21c5-498d-8998-6890da5987ab',
 '4bd2e119-a059-4209-b531-9f02ce433d3a',
 '7e0586eb-81b2-4de6-85df-94184402c2da',
 '607af001-eaa5-4c3f-99fe-2fc9b210e7d4',
 '0e652802-4a54-4363-aa0b-4710abf4edc8',
 '7782934b-98d8-4c08-9a6c-d0f49f30faa2',
 '044337b5-d261-4bcf-8ca9-0f90ebf9f1e7',
 'df899004-dc83-4b25-80a7-2eef62315cb6',
 '409dbe65-e37a-4682-9748-6f89032b9a8c',
 '5bd5b218-97bc-46a9-bb7c-87e7ad60150c',
 '030a01d1-4583-4a75-b333-df48ed1f536b',
 '0b218164-88cb-435f-8727-1a2575bc015e',
 '70e80fd7-d29d-

In [31]:
vs.save_local("/data/dataiku/dss_data/knowledge-banks/TEST_UPLOAD_RAG/IPfgEibh/")

For more information on the Dataiku Python API relating to LLM Mesh, see here: https://developer.dataiku.com/latest/concepts-and-examples/llm-mesh.html